We adopt dynamical (binary search) prices for impression vector as a probe, namely the price = budget cutoff for the untargeted marget segment.
If we win the advertising auction, then we shade down the price; If we lose the advertising auction, we scale up the price.
The price vector (8, ) at the end of the n-th day is fed into the LSTM neural network.
The open campaigns are revealed to us. We maintain a container which keeps track of those open campaigns on the (n+1)-th day and can be represented as a campaign vector (26, ). This campaign vector is also fed into the LSTM neural network. The goal of the LSTM neural network is to predict the hidden campaigns on the (n+1)-th day. The output of the LSTM neural network is a vector of softmax function (26, ), representing the probability of each campaign. Then we can reconstruct the campaigns with reach = budget = probability * 10000 / 3.
The LSTM neural network is pretrained but will also be trained online. 

In [1]:
from typing import List, Set, Dict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import LBFGS, NAdam
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [13]:
class CampaignPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CampaignPredictor, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # LSTM with specified number of layers and neurons
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

        # Xavier normal initialization
        self.init_weights()

    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_normal_(param.data)
            elif 'weight_hh' in name:
                nn.init.xavier_normal_(param.data)
        nn.init.xavier_normal_(self.fc.weight)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate the LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)  # Softmax is applied later in the training loop


In [14]:
class CampaignDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence = self.sequences[index]
        # Assuming each sequence is a tuple (features, target)
        features = torch.tensor(sequence[0], dtype=torch.float32)
        target = torch.tensor(sequence[1], dtype=torch.float32)
        return features, target

# train_loader = ...
# test_loader = ...

In [ ]:
input_dim = 34
output_dim = 26
hidden_dim = 100
layer_dim = 3
NAdam_iter = 5000
LBFGS_iter = 3000
batch_size = 1000

criterion = nn.BCEWithLogitsLoss()
NAdam_optimizer = NAdam(model.parameters(), lr=0.001)
LBFGS_optimizer = LBFGS(model.parameters())

In [15]:
def train(model, optimizer, train_loader, epochs, device):
    model.train()
    model.to(device)

    for epoch in range(epochs):
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(features.float())
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            def closure():
                optimizer.zero_grad()
                outputs = model(features.float())
                loss = criterion(outputs, labels)
                loss.backward()
                return loss
        
            optimizer.step(closure)

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    with torch.no_grad():
        total_credits = 0  # Total credits earned, including partial credits
        total_possible = 0  # Total possible credits (each label element counts as one credit)

        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features.float())
            
            # Multiply probabilities by 10 and round to nearest integer
            predicted = (outputs * 10).round()

            # Compute the absolute difference between predicted and true labels
            difference = torch.abs(predicted - labels)

            # Full credit for exact matches, half credit for off by one
            credits = (difference == 0).float() + (difference == 1).float() * 0.5
            
            total_credits += credits.sum().item()
            total_possible += labels.numel()  # Number of label elements processed

        # Calculate the percentage of total credits earned out of total possible credits
        accuracy_percentage = 100 * total_credits / total_possible
        print(f'Accuracy (with partial credits): {accuracy_percentage:.2f}%')

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CampaignPredictor(input_size, hidden_size, num_layers, output_size)
train(model, NAdam_optimizer, NAdam_iter, train_loader, epochs, device=device)
train(model, LBFGS_optimizer, LBFGS_iter, train_loader, epochs, device=device)
evaluate(model, test_loader, device=device)

model(features)


NameError: name 'train_loader' is not defined